In [140]:
import pandas as pd
import numpy as np
import re
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from os import path
import os
import config
import pickle

In [75]:
def query_spotify_id(search, cid, secret):
    
    search = search.replace(" ", "+")
    
    client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
    sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)
    
    query = sp.search(search)
    
    result = {}
    result['id'] = query['tracks']['items'][0]['id']
    result['artist'] = query['tracks']['items'][0]['artists'][0]['name']
    result['title'] = query['tracks']['items'][0]['name']
    
    return result

In [158]:
ss = 
sc = 

In [203]:
query = "lil yachty t.d"

In [204]:
result = query_spotify_id(query, sc, ss)

In [205]:
result

{'id': '0gO3E51A3fqp0ypJ26QbVP',
 'artist': 'Lil Yachty',
 'title': 'T.D (Lil Yachty & Tierra Whack feat. A$AP Rocky & Tyler, The Creator)'}

In [206]:
def get_song_features(song_id, cid, secret):
    client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
    sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)
    
    audio_feature = sp.audio_features(song_id)[0]
    
    return audio_feature

In [207]:
features = get_song_features(result['id'], sc, ss)
features

{'danceability': 0.784,
 'energy': 0.632,
 'key': 10,
 'loudness': -6.083,
 'mode': 0,
 'speechiness': 0.347,
 'acousticness': 0.156,
 'instrumentalness': 0,
 'liveness': 0.605,
 'valence': 0.746,
 'tempo': 124.603,
 'type': 'audio_features',
 'id': '0gO3E51A3fqp0ypJ26QbVP',
 'uri': 'spotify:track:0gO3E51A3fqp0ypJ26QbVP',
 'track_href': 'https://api.spotify.com/v1/tracks/0gO3E51A3fqp0ypJ26QbVP',
 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/0gO3E51A3fqp0ypJ26QbVP',
 'duration_ms': 232839,
 'time_signature': 4}

In [208]:
def create_modeling_features(features):
    key_dict = {0:'C', 1:'Db', 2: 'D', 3:'Eb', 4:'E', 5:'F', 6:'Gb', 7:'G', 8:'Ab', 9:'A', 10:'Bb', 11:'B'}
    mode_dict = {1:'major', 0:'minor'}
    feature_names = ['danceability', 'energy','loudness', 'acousticness', 'speechiness',
                'instrumentalness', 'liveness', 'valence', 'tempo','duration_ms',
                'key_A', 'key_Ab', 'key_B', 'key_Bb', 'key_C', 'key_D', 'key_Db', 'key_E', 
                'key_Eb', 'key_F', 'key_G', 'key_Gb','mode_major']
    
    # Get key name, convert to binary feature
    key = key_dict[features['key']]
    key_feature_name = "key_" + key
    features.update( {key_feature_name : 1} )
    
    # Add mode_major feature
    features.update({"mode_major": features['mode']})
    
    # Create dictionary with modeling features
    model_features = {}
    for f in feature_names:
        if f in list(features.keys()):
            model_features[f] = features[f]
        else:
            model_features[f] = 0
            
    # Return modeling features as single row DataFrame
    return pd.DataFrame([model_features])

In [209]:
features_df = create_modeling_features(features)

In [210]:
model = pickle.load(open( "/Users/briancai/Drive/NU/Q3/AVC/litness_test/2020-MSIA423-Cai-Litness-Test/model/random_forest_classifier.pkl", "rb" ) )

In [211]:
features_df

,danceability,energy,loudness,acousticness,speechiness,instrumentalness,liveness,valence,tempo,duration_ms,...,key_Bb,key_C,key_D,key_Db,key_E,key_Eb,key_F,key_G,key_Gb,mode_major
0,0.784,0.632,-6.083,0.156,0.347,0,0.605,0.746,124.603,232839,...,1,0,0,0,0,0,0,0,0,0


In [212]:
model.predict_proba(features_df)[0][1]

0.8655860400263706

In [1]:
test = {"tiff": "bagels", "brian": "popsicle"}

In [2]:
test

{'tiff': 'bagels', 'brian': 'popsicle'}

In [5]:
test['tiff'] = "olives"

In [6]:
test

{'tiff': 'olives', 'brian': 'popsicle'}